# Задача B: Алгоритм Форда – Беллмана

Источник: [лаба про пути](http://cs.mipt.ru/algo/lessons/sem_2/09.graph_deykstra.html).

Дан ориентированный взвешенный граф и номер стартовой вершины.
Вершины нумеруются с нуля.
Необходимо определить кратчайшие расстояния от стартовой вершины до остальных. 
 
## Формат входных данных

На вход программе в первой строке подается три числа через пробел:

* 2 ≤ `n` ≤ 1000 — число вершин в графе
* 1 ≤ `m` ≤ 1000 — число рёбер
* `s` — номер начальной вершины

В следующих `m` строках задаются рёбра.
Ребро задаётся тремя числами через пробел:

* начало ребра
* конец ребра
* вес ребра

Вес ребра — целое число, по модулю не превышаюшее 1000. 

## Формат выходных данных

Необходимо вывести строку из n элементов через пробел — расстояния до вершин из заданной.
Если расстояние до какой-то вершины не определено, то выведите вместо этого расстояния строку "UDF". 

## Примеры

Пример 1:
```
Вход:

2 1 0
0 1 5

Выход:

0 5
```

Пример 2:
```
Вход:

4 5 0
0 1 10
0 2 40
1 2 15
0 3 20
3 1 5

Выход:

0 10 25 20
```

## Решение

In [1]:
from typing import Dict

### Считывание данных ("сырой" граф)

In [2]:
V, E, start = [int(n) for n in input().split()]

edges = []

for _ in range(E):
    v1, v2, w = [int(n) for n in input().split()]
    edges.append((v1, v2, w))

4 5 0
0 1 10
0 2 40
1 2 15
0 3 20
3 1 5


### "Причёсывание" графа

с прицелом на последующего Беллмана – Форда для поиска расстояний (ради этого хэш)

In [3]:
class Node:
    def __init__(self, value):
        self.value = value
        self.descendants: Dict[Node, int] = dict()

    def __eq__(self, other):
        return self.value == other.value
    
    def __hash__(self):
        return hash(self.value)

In [4]:
nodes = [
    Node(i) for i in range(V)
]

for edge in edges:
    v1, v2, w = edge
    nodes[v1].descendants[nodes[v2]] = w  # пользуемся тем, что вершины пронумерованы

In [5]:
def find_distances_bfs(root):
    distances = {
        n: float('+inf') for n in nodes
    }
    distances[root] = 0

    wave = [root]
    queue = [wave]
    
    max_step = V - 1  # если циклов нет, то во всех кратчайших путях будет максимум V - 1 ребро
    
    for current_step in range(1, max_step + 1):
        # Start is also included in wave (wave = start + all steps)
        assert all(len(w) == current_step for w in queue)

        step_forward_bfs(queue, distances)
    
    if len(queue) == 0:  # нет циклов отрицательной длины
        return distances

    find_negative_cycles_bfs(queue, distances)

    return distances


def step_forward_bfs(queue, distances):
    if len(queue) == 0:
        return
    
    assert all(len(wave) == len(first_wave := queue[0]) for wave in queue)

    num_waves_to_expand = len(queue)  # all new waves go to the next round of BF

    for _ in range(num_waves_to_expand):
        wave = queue.pop(0)  # it's not very nice to pop from list's start, but whatever
        root = wave[-1]

        for d, w in root.descendants.items():
            if distances[d] > (new_distance := distances[root] + w):
                if d in wave and new_distance < 0:
                    distances[d] = float('-inf')
                else:
                    distances[d] = new_distance

                new_wave = list(wave) + [d]
                queue.append(new_wave)


def find_negative_cycles_bfs(queue, distances):        
    frozen_distances = dict(distances)  # copy
    step_forward_bfs(queue, distances)
    
    if distances == frozen_distances:  # нет циклов отрицательной длины
        assert len(queue) == 0
        
        return distances

    assert len(queue) > 0
        
    while len(queue) > 0:  # it will be zero after some time
                           # (because updates will stop -- all updated distances are set equal to -inf)
        updated_nodes = [wave[-1] for wave in queue]
        
        for node in updated_nodes:
            distances[node] = float('-inf')

        assert all(distances[n] < frozen_distances[n] for n in updated_nodes)

        step_forward_bfs(queue, distances)

In [6]:
distances = find_distances_bfs(nodes[start])

for n, d in distances.items():
    if d in [float('+inf'), float('-inf')]:
        distances[n] = 'UDF'

print(*distances.values())

0 10 25 20
